In [38]:
import urllib.request
from bs4 import BeautifulSoup
import re
import json
import time
from socket import timeout

In [39]:
def generate_classification_links():
    sourcelink = 'http://www.mftd.org/index.php?action=atu&act=select&atu='
    file = open('generate_classification_links.txt', 'w', encoding = 'utf-8')
    links = []
    for i in range (1, 2400):
        links.append(sourcelink + str(i) + '\n')
    js = json.dumps(links, ensure_ascii = False)
    file.write(js)

In [3]:
generate_classification_links()

In [41]:
def open_link(link):
    try:
        a = urllib.request.urlopen(link)
        file = a.read()
    except timeout:
        file = None
    return file

In [5]:
def database_with_ids():
    time.sleep(20)
    file_r = open('generate_classification_links.txt', 'r', encoding = 'utf-8')
    file_r = json.loads(file_r.read())
    file_w = open('database_with_ids.txt', 'w', encoding = 'utf-8')
    dat = {}
    COUNT = 0
    for line in (file_r):
        everything = []
        soup = boil_some_soup(line)
        try:
            number, name = number_name(soup)
            hier = get_hier(soup)
            ids = get_ex_ids(soup)
            everything.append(name)
            everything.append(hier)
            everything.append(ids)
            dat[number] = everything
        except ValueError:
            print(soup.h2.text)    
        except AttributeError:
            print(str(COUNT) + ' missin ' + line)
            COUNT +=1
    js = json.dumps(dat, ensure_ascii = False)
    file_w.write(js)

In [1]:
#database_with_ids()

In [6]:
def boil_some_soup(link):
    soup = ''
    try:
        some_text = open_link(link)
        soup = BeautifulSoup(some_text)
    except UnicodeEncodeError:
        print(link)
    except timeout:
        print(link)
    return soup

In [7]:
def number_name(soup):
    name = soup.h2.text
    number, name = re.split(': ', name)
    return (number, name)

In [8]:
def get_hier(soup):
    something = soup.find_all('div')
    something = something[1]
    ps = something.find_all('p')
    hier = []
    for p in range(3):
        p = re.match(r'<p>(.*)<a href="index\.php\?action=atu&amp;act=range&amp;id=(.*)', str(ps[p]))
        try:
            p = p.group()
            p = re.sub(r'<p>(.*)<a href="index\.php\?action=atu&amp;act=range&amp;id=(.*)">(.*)<\/a> ', '', p)
            p = re.sub(r'</p>', '', p)
            hier.append(p)
        except AttributeError:
            break
    return hier

In [9]:
def get_ex_ids(soup):
    table_examples = soup.table
    ids = re.findall(r'<a href="index\.php\?action=story&amp;act=select&amp;id=(.*)">', str(table_examples))
    return ids

в database_with_ids.txt структура такая:  
{номер мотива: [название мотива,[1, 2, 3], [id примеров]]}  
хранится в джесоне  
айтемов получилось чуть меньше тысячи  


потерялись:  
613  
925  
969  
1070  
1153  
1183  
1453  
1577  
1624  
1738  
1812  
1836A  

если решим их не терять с ними надо все тоже самое сделать –– только как-то по-другому парсить number_name(), потому что моя функция сплитит номер и имя по двоеточию, а у них оно встречается в имени  

?как хранить сказки:
{номер мотива: [название мотива,[1, 2, 3], [[пример,шапка],[пример,шапка],...]}

In [10]:
def read_json(name):
    file_r = open(name, 'r', encoding = 'utf-8')
    file_r = json.loads(file_r.read())
    return (file_r)

In [34]:
def get_tales():
    ids = read_json('database_with_ids.txt')
    all_the_things = []
    counter = -1
    for this1 in ids:
        some_things = []
        counter += 1
        this = ids[this1]
        this = this[2]
        for that in this:
            base = 'http://www.mftd.org/index.php?action=story&act=select&id='
            time.sleep(2)
            soup = boil_some_soup(base + str(that))
            if soup == None:
                break
            else:
                try:
                    name = get_tale_name(soup)
                    some_things.append(that)
                    some_things.append(name)
                    try:    
                        tables = get_table(soup)
                        some_things.append(tables)
                    except AttributeError:
                        print (soup.prettify)
                    all_the_things.append(some_things)
                except AttributeError:
                    print(soup)
    try:
        file_w = open('database_with_tables.txt', 'w', encoding = 'utf-8')
        js = json.dumps(all_the_things, ensure_ascii = False)
        file_w.write(js)
    except TypeError:
        print (all)

In [36]:
def get_tale_name(soup):
    something = soup.find_all('h1')
    name = something[3].text
    return name

In [13]:
def get_table(soup):
    something = soup.find_all('div')
    something = something[1]
    tables = something.find_all('div', class_='infobox')
    return str(tables)

In [42]:
get_tales()

http://www.mftd.org/index.php?action=story&act=select&id=3181">Rødrev og Askeladden</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=3873">A Bicha de Sete  Cabeças</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=3476">Țugulea, fiul unchiașului și al mătușei</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=3177">Gutten som gjorde seg til løve, falk og maur</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=3891">Pohádka o perníkové chaloupce</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=3784">El zurrón que cantaba</a></td><td></td></tr><tr><td style="padding-right: 10px;

http://www.mftd.org/index.php?action=story&act=select&id=6016">Lu Re d'Anìmmulu</a></

^ ссылки это то, что мы потеряли из-за кодировки

ЧТО ТЕПЕРЬ?  
- джесон открывается функцией read_json(name)  
    в нем лежит список [[имя_сказки][id_сказки][таблица_с_метаданными_таблица_с_переводами]]  
    id сказки – это то что мы добавляем к базовой ссылке чтобы попасть на старницу с инфой про сказку  
- таблицы хранятся стрингами, поэтому их надо снова конвертировать в суп (ну или парсить регулярками)  